In [2]:
import pandas as pd

In [3]:
import threading
import time
import openai
from threading import Lock
from langchain_community.embeddings import OpenAIEmbeddings
# from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
import json
from langchain_community.vectorstores import Chroma
import pandas as pd
import sys

In [4]:
pwd

'c:\\Users\\Pyrack\\Desktop\\dorisjv\\dorisclinicaltrials\\Notebooks'

In [39]:
df = pd.read_csv("FinalCTTrialsDF_Full_w_ContactInfo_LocList_testing.csv")

In [40]:
df_1 = pd.read_csv("FinalCTTrialsDF_P1_w_ContactInfo_LocList.csv")

In [7]:
df_1

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT,LOCATIONS
0,0,0,0,0,NCT03275311,Evolution of the Therapeutic Care in Metastati...,https://www.clinicaltrials.gov/study/NCT03275311,Recruiting,Metastatic Breast Cancer,July 2014,December 2024,18 Years||Adult|Older Adult,Eligibility includes patients 18 years or olde...,The primary outcome measure is to describe the...,Secondary outcomes focus on the impact of ther...,The study does not specify particular interven...,"{'Name': None, 'Organization': None, 'Email': ...","[{'Location Facility': ""Institut de Cancérolog..."
1,20,20,20,20,NCT06145399,A Study of 1 8F-FDHT PET/MRIScans in Women Wit...,https://www.clinicaltrials.gov/study/NCT06145399,Recruiting,"Breast Cancer, Breast Carcinoma, Metastatic Br...","October 24, 2023","October 24, 2026",18 Years||Adult|Older Adult,Eligibility includes individuals aged ≥ 18 yea...,The primary outcome is to correlate 18F-FDHT u...,No secondary outcomes were provided.,Participants with histologically confirmed AR+...,"{'Name': None, 'Organization': None, 'Email': ...",[{'Location Facility': 'Memorial Sloan Ketteri...
2,21,21,21,21,NCT05496101,CLI and FAR for Intraoperative Margin Assessment,https://www.clinicaltrials.gov/study/NCT05496101,Recruiting,Breast Cancer Female,"November 21, 2022","December 31, 2024",18 Years||Adult|Older Adult,Eligibility includes female subjects ≥18 years...,The primary outcome focuses on the margin stat...,Secondary outcomes include the agreement betwe...,The intervention involves the use of the Light...,"{'Name': None, 'Organization': None, 'Email': ...","[{'Location Facility': ""Guy's and St Thomas NH..."
3,22,22,22,22,NCT05935384,SIBYL: obServation of Therapy Response With lI...,https://www.clinicaltrials.gov/study/NCT05935384,Recruiting,"Non-small Cell Lung Cancer, Colorectal Cancer,...","October 25, 2023","August 30, 2029",18 Years||Adult|Older Adult,Eligibility includes patients treated with fir...,The primary outcome is the sensitivity of circ...,Secondary outcomes include RECIST v1.1 respons...,The intervention involves the Guardant360 diag...,"{'Name': None, 'Organization': None, 'Email': ...",[{'Location Facility': 'Orchard Healthcare Res...
4,23,23,23,23,NCT05281237,Avecure Flexible Microwave Ablation Probe For ...,https://www.clinicaltrials.gov/study/NCT05281237,Recruiting,"Stage I - II Primary Lung Cancer, Stage I Lung...","June 1, 2022","June 30, 2025",22 Years||Adult|Older Adult,Eligibility includes individuals with Stage I ...,The primary outcomes focus on the feasibility ...,Secondary outcomes involve examining histologi...,The intervention involves the use of the AveCu...,"{'Name': None, 'Organization': None, 'Email': ...",[{'Location Facility': 'Beth Israel Deaconess ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,17887,17887,17887,17887,NCT05006131,Pancreatic Cancer Screening for At-risk Indivi...,https://www.clinicaltrials.gov/study/NCT05006131,Recruiting,"Pancreatic Cancer, Adult","July 10, 2020","July 10, 2026",18 Years|90 Years|Adult|Older Adult,Eligibility includes individuals indicated for...,The primary outcome is the proportion of scree...,Secondary outcomes include the proportion of s...,The intervention involves screening for pancre...,"{'Name': None, 'Organization': None, 'Email': ...",[{'Location Facility': 'Beth Israel Deaconess ...
1096,17888,17888,17888,17888,NCT04679064,Trial on NIraparib-TSR-042 (Dostarlimab) vs Ph...,https://www.clinicaltrials.gov/study/NCT04679064,Recruiting,Ovarian Cancer,"December 1, 2020","January 1, 2025",18 Years||Adult|Older Adult,"Eligibility includes recurrent ovarian, Fallop...",The primary outcome measure is overall surviva...,Secondary outcomes include progression-free su...,Interventions include a choice of standard che...,"{'Name': None, 

In [8]:
df

,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT
0,0,NCT05664971,JS004 Combined With Toripalimab and With Stand...,https://www.clinicaltrials.gov/study/NCT05664971,Recruiting,Advanced Lung Cancer,[{'Location Facility': 'Shanghai Chest Hospita...,"February 9, 2023","November 17, 2024",18 Years||Adult|Older Adult,Eligible participants include males or females...,The primary outcomes include the incidence of ...,Secondary outcomes encompass the duration of r...,The study involves multiple cohorts receiving ...,"{'Name': None, 'Organization': None, 'Email': ..."
1,1,NCT05902520,Adoptive Cell Therapy Using Cancer Specific CD...,https://www.clinicaltrials.gov/study/NCT05902520,Recruiting,"HNSCC, Melanoma, Gynecologic Cancer, Colorecta...",[{'Location Facility': 'Providence Portland Me...,"June 19, 2023","May 19, 2026",18 Years||Adult|Older Adult,Eligibility includes adults over 18 with histo...,The primary outcome is to assess the safety of...,Secondary outcomes include evaluating the effi...,The study involves two experimental arms. Both...,"{'Name': None, 'Organization': None, 'Email': ..."
2,2,NCT03917043,APG-2449 in Patients With Advanced Solid Tumors,https://www.clinicaltrials.gov/study/NCT03917043,Recruiting,"Advanced Solid Cancer, Non Small Cell Lung Can...",[{'Location Facility': 'Beijing Cancer Hospita...,"May 27, 2019",February 2025,18 Years||Adult|Older Adult,Eligibility includes patients with advanced tu...,The primary outcomes are to determine the maxi...,Secondary outcomes include measuring the maxim...,The intervention involves administering APG-24...,"{'Name': None, 'Organization': None, 'Email': ..."
3,3,NCT05674422,GEMCAD-REVEAL STUDY - Circulating Tumor DNA as...,https://www.clinicaltrials.gov/study/NCT05674422,Recruiting,Patients With Locally Advanced Rectal Cancer,[{'Location Facility': 'Hospital Germans Trias...,"May 26, 2023",July 2026,18 Years||Adult|Older Adult,Eligible participants are legally capable indi...,The primary outcome measure is the detection o...,Secondary outcomes include the classification ...,The intervention involves total neoadjuvant th...,"{'Name': None, 'Organization': None, 'Email': ..."
4,4,NCT06249854,Efficacy and Safety of Combination Therapy Wit...,https://www.clinicaltrials.gov/study/NCT06249854,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Korea University Ansan...,"February 8, 2024","December 31, 2025",19 Years||Adult|Older Adult,Eligibility includes adults aged 19 or older w...,The primary outcome measure is Progression-Fre...,Secondary outcomes include Disease Control Rat...,The study involves two groups: one receiving P...,"{'Name': None, 'Organization': None, 'Email': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,3831,NCT05006131,Pancreatic Cancer Screening for At-risk Indivi...,https://www.clinicaltrials.gov/study/NCT05006131,Recruiting,"Pancreatic Cancer, Adult",[{'Location Facility': 'Beth Israel Deaconess ...,"July 10, 2020","July 10, 2026",18 Years|90 Years|Adult|Older Adult,Eligibility includes individuals indicated for...,The primary outcome is the proportion of scree...,Secondary outcomes include the proportion of s...,The intervention involves screening for pancre...,"{'Name': None, 'Organization': None, 'Email': ..."
3832,3832,NCT04679064,Trial on NIraparib-TSR-042 (Dostarlimab) vs Ph...,https://www.clinicaltrials.gov/study/NCT04679064,Recruiting,Ovarian Cancer,[{'Location Facility': 'Azienda Ospedaliera Sp...,"December 1, 2020","January 1, 2025",18 Years||Adult|Older Adult,"Eligibility includes recurrent ovarian, Fallop...",The primary outcome measure is overall surviva...,Secondary outcomes include progression-free su...,Interventions include a choice of standard che...,"{'Name': None, 'Organization': None, 'Email': ..."
3833,3833,NCT04491864,DWIBS-MRI: An Adjunct to the Trad

In [9]:
lung_cancer_df = df[df['CONDITIONS'].str.contains('Lung Cancer', case=False)]

In [10]:
df.columns

Index(['Unnamed: 0', 'NCT_NUMBER', 'STUDY_TITLE', 'STUDY_URL', 'STUDY_STATUS',
       'CONDITIONS', 'LOCATIONS', 'START_DATE', 'COMPLETION_DATE', 'AGE',
       'ELIGIBILITY_CRITERIA', 'PRIMARY_OUTCOMES', 'SECONDARY_OUTCOMES',
       'INTERVENTIONS', 'POINT_OF_CONTACT'],
      dtype='object')

In [11]:
df_1.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0',
       'NCT_NUMBER', 'STUDY_TITLE', 'STUDY_URL', 'STUDY_STATUS', 'CONDITIONS',
       'START_DATE', 'COMPLETION_DATE', 'AGE', 'ELIGIBILITY_CRITERIA',
       'PRIMARY_OUTCOMES', 'SECONDARY_OUTCOMES', 'INTERVENTIONS',
       'POINT_OF_CONTACT', 'LOCATIONS'],
      dtype='object')

In [12]:
lung_cancer_df

,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT
0,0,NCT05664971,JS004 Combined With Toripalimab and With Stand...,https://www.clinicaltrials.gov/study/NCT05664971,Recruiting,Advanced Lung Cancer,[{'Location Facility': 'Shanghai Chest Hospita...,"February 9, 2023","November 17, 2024",18 Years||Adult|Older Adult,Eligible participants include males or females...,The primary outcomes include the incidence of ...,Secondary outcomes encompass the duration of r...,The study involves multiple cohorts receiving ...,"{'Name': None, 'Organization': None, 'Email': ..."
1,1,NCT05902520,Adoptive Cell Therapy Using Cancer Specific CD...,https://www.clinicaltrials.gov/study/NCT05902520,Recruiting,"HNSCC, Melanoma, Gynecologic Cancer, Colorecta...",[{'Location Facility': 'Providence Portland Me...,"June 19, 2023","May 19, 2026",18 Years||Adult|Older Adult,Eligibility includes adults over 18 with histo...,The primary outcome is to assess the safety of...,Secondary outcomes include evaluating the effi...,The study involves two experimental arms. Both...,"{'Name': None, 'Organization': None, 'Email': ..."
2,2,NCT03917043,APG-2449 in Patients With Advanced Solid Tumors,https://www.clinicaltrials.gov/study/NCT03917043,Recruiting,"Advanced Solid Cancer, Non Small Cell Lung Can...",[{'Location Facility': 'Beijing Cancer Hospita...,"May 27, 2019",February 2025,18 Years||Adult|Older Adult,Eligibility includes patients with advanced tu...,The primary outcomes are to determine the maxi...,Secondary outcomes include measuring the maxim...,The intervention involves administering APG-24...,"{'Name': None, 'Organization': None, 'Email': ..."
4,4,NCT06249854,Efficacy and Safety of Combination Therapy Wit...,https://www.clinicaltrials.gov/study/NCT06249854,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Korea University Ansan...,"February 8, 2024","December 31, 2025",19 Years||Adult|Older Adult,Eligibility includes adults aged 19 or older w...,The primary outcome measure is Progression-Fre...,Secondary outcomes include Disease Control Rat...,The study involves two groups: one receiving P...,"{'Name': None, 'Organization': None, 'Email': ..."
7,7,NCT05777603,Study of Aerosolized Antibiotics and Pembroliz...,https://www.clinicaltrials.gov/study/NCT05777603,Recruiting,Advanced Non Small Cell Lung Cancer,[{'Location Facility': 'National Institutes of...,"February 18, 2024","June 1, 2025",18 Years|120 Years|Adult|Older Adult,Eligibility includes adults (>=18 years) with ...,The primary outcome is to evaluate the safety ...,Secondary outcomes include determining the rec...,The intervention consists of an experimental a...,"{'Name': None, 'Organization': None, 'Email': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809,3809,NCT06091943,Study to Evaluate the Bioavailability of Tisle...,https://www.clinicaltrials.gov/study/NCT06091943,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Peking Union Medical C...,"November 16, 2023","March 31, 2026",18 Years||Adult|Older Adult,Eligibility includes individuals able to conse...,Primary outcomes include pharmacokinetic measu...,Secondary outcomes focus on additional pharmac...,The study involves two experimental arms. Part...,"{'Name': None, 'Organization': None, 'Email': ..."
3811,3811,NCT04892472,"EF-36/Keynote B36: A Pilot, Randomized, Open-l...",https://www.clinicaltrials.gov/study/NCT04892472,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Central Alabama Resear...,"July 12, 2021",December 2026,22 Years||Adult|Older Adult,Eligibility includes individuals with a confir...,The primary outcome measure is Progression Fre...,Secondary outcomes include Overall Survival (O...,The study involves two experimental arms. Arm ...,"{'Name': None, 'Organization': None, 'Email': ..."
3812,3812,NCT05236608,A Study to Evaluate t

In [13]:
output = {"stage" : "stage",
          "age" : "age",
          "gender" : "gender" }

In [14]:
system_prompt = """You are helpful assistant to give stage of cancer ,age and gender from given senetence and give output in json format as {output} """

In [15]:
ELIGIBILITY_CRITERIA = df["ELIGIBILITY_CRITERIA"][2]

In [16]:
df["ELIGIBILITY_CRITERIA"][2]

'Eligibility includes patients with advanced tumors such as non-small cell lung cancer with ALK/ROS1 gene fusion, malignant pleural mesothelioma, esophageal cancer, and ovarian cancer. Specific cohorts for expansion stage include patients with non-small cell lung cancer who have progressed on or are intolerant to second-generation ALK TKI therapy or any ROS1 TKI therapy, and those with ALK/ROS1 fusion gene positive non-small cell lung cancer without prior TKI treatment. Requirements include ECOG Performance Status ≤ 1, life expectancy ≥ 3 months, at least 1 measurable lesion per RECIST 1.1, adequate organ function, controlled brain metastases, and negative pregnancy test for women of childbearing age. Exclusions include concurrent anti-cancer therapy, recent TKI therapy, unresolved toxicities, swallowing difficulties, significant cardiovascular disease, recent surgery, active infections, known allergies to study drug, pregnant or breastfeeding women, and use of certain CYP inhibitors o

In [17]:
response = openai.chat.completions.create(
            model='gpt-4-0125-preview', 
            temperature=0,
            messages=[
                {"role":"system", "content":system_prompt},
                {"role":"user", "content":f""" Extract the stage of cancer, age and gender from given input {ELIGIBILITY_CRITERIA} and give output in {output} format, stricrtly specify gender as male or female """},
            ],
            max_tokens = 1024,
            response_format={ "type": "json_object" }
            
        )
      

In [18]:
response.choices[0].message.content

'{"stage": "advanced", "age": "not specified", "gender": "not specified"}'

In [19]:
df["AGE"].unique()

array(['18 Years||Adult|Older Adult', '19 Years||Adult|Older Adult',
       '18 Years|75 Years|Adult|Older Adult',
       '18 Years|120 Years|Adult|Older Adult',
       '18 Years|80 Years|Adult|Older Adult',
       '21 Years||Adult|Older Adult',
       '45 Years|65 Years|Adult|Older Adult', '65 Years||Older Adult',
       '18 Years|99 Years|Adult|Older Adult',
       '20 Years|75 Years|Adult|Older Adult', '||Child|Adult|Older Adult',
       '18 Years|90 Years|Adult|Older Adult',
       '18 Years|70 Years|Adult|Older Adult', '25 Years|60 Years|Adult',
       '1 Month||Child|Adult|Older Adult', '40 Years|60 Years|Adult',
       '18 Years|39 Years|Adult', '45 Years|49 Years|Adult',
       '19 Years|85 Years|Adult|Older Adult',
       '18 Years|65 Years|Adult|Older Adult',
       '18 Years|100 Years|Adult|Older Adult',
       '12 Years||Child|Adult|Older Adult', '50 Years||Adult|Older Adult',
       '18 Years|66 Years|Adult|Older Adult',
       '19 Years|70 Years|Adult|Older Adult',
      

In [20]:
month_df = df[df['AGE'].str.contains('month', case=False)]

In [21]:
month_df["AGE"]

79              1 Month||Child|Adult|Older Adult
586               12 Months|30 Years|Child|Adult
1566          18 Months||Child|Adult|Older Adult
2467          18 Months||Child|Adult|Older Adult
2507              12 Months|30 Years|Child|Adult
3557    1 Month|99 Years|Child|Adult|Older Adult
Name: AGE, dtype: object

In [22]:
df_month = month_df.copy()

In [23]:
df_month

,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT
79,79,NCT02595957,Genomic Services Research Program,https://www.clinicaltrials.gov/study/NCT02595957,Recruiting,"Colon Cancer, Breast Cancer",[{'Location Facility': 'National Institutes of...,"September 16, 2014","December 31, 2028",1 Month||Child|Adult|Older Adult,Eligibility includes English- or Spanish-speak...,The study aims to assess affective responses a...,Not specified.,The study involves two groups: 'Cascade Testin...,"{'Name': None, 'Organization': None, 'Email': ..."
586,586,NCT02574728,Sirolimus in Combination With Metronomic Chemo...,https://www.clinicaltrials.gov/study/NCT02574728,Recruiting,Cancer,"[{'Location Facility': ""Phoenix Children's Hos...",June 2015,January 2024,12 Months|30 Years|Child|Adult,Eligibility includes participants with relapse...,Primary outcomes focus on the change in radiog...,The secondary outcome is the number of adverse...,"Participants will receive oral sirolimus, cele...","{'Name': None, 'Organization': None, 'Email': ..."
1566,1566,NCT04740697,Study to Evaluate the Impact of the Use of Alt...,https://www.clinicaltrials.gov/study/NCT04740697,Recruiting,Breast Cancer,[{'Location Facility': 'Institut Universitaire...,"February 12, 2021",February 2024,18 Months||Child|Adult|Older Adult,Eligibility includes individuals aged ≥ 18 yea...,The primary outcome is the rate of patients wh...,Secondary outcomes include the rate of patient...,Intervention involves patients with localized ...,"{'Name': None, 'Organization': None, 'Email': ..."
2467,2467,NCT05783908,Impact of Non-magnetic Screen Respiratory Self...,https://www.clinicaltrials.gov/study/NCT05783908,Recruiting,"Patient With Radiotherapy (Linac MRI), Mobile ...",[{'Location Facility': 'Centre Georges-Françoi...,"October 16, 2023","September 27, 2025",18 Months||Child|Adult|Older Adult,Eligibility includes individuals at least 18 y...,The primary outcome is to evaluate the effect ...,Secondary outcomes include measuring patient r...,The study involves two arms: one receiving rad...,"{'Name': None, 'Organization': None, 'Email': ..."
2507,2507,NCT05569512,Uproleselan With Pre-Transplant Conditioning i...,https://www.clinicaltrials.gov/study/NCT05569512,Recruiting,"Acute Myeloid Leukemia, Pediatric Cancer",[{'Location Facility': 'University of Alabama ...,"October 6, 2022","December 1, 2028",12 Months|30 Years|Child|Adult,Eligibility includes individuals aged 12 month...,Primary outcomes focus on determining the reco...,Secondary outcomes include uproleselan pharmac...,Participants will receive intravenous (IV) upr...,"{'Name': None, 'Organization': None, 'Email': ..."
3557,3557,NCT03896958,The PIONEER Initiative: Precision Insights On ...,https://www.clinicaltrials.gov/study/NCT03896958,Recruiting,"Cancer, All Types, Cancer of Liver, Cancer of ...","[{'Location Facility': 'Specicare', 'Location ...","March 21, 2019","March 12, 2024",1 Month|99 Years|Child|Adult|Older Adult,Inclusion criteria include all patients diagno...,The study focuses on three primary outcomes ov...,No secondary outcomes were provided.,No specific interventions were detailed.,"{'Name': None, 'Organization': None, 'Email': ..."


In [24]:
import re

# Function to convert age format
def convert_age_format(age_info):
    # Split the age info by "|" and extract numerical values
    ages = re.findall(r'\d+', age_info)
    
    # Convert the numerical values to integers
    ages = [int(age) for age in ages if age]  # Filter out empty strings
    
    # Check if there are any age values
    if ages:
        # If there are age values, join them with "years |" separator
        formatted_ages = " | ".join(f"{age} years" for age in ages)
    else:
        # If no age values are found, return the original age_info
        formatted_ages = age_info
    
    return formatted_ages

In [25]:
import re

# Function to convert age format
def convert_age_format_months(age_info):
    # Split the age info by "|" and extract numerical values
    ages = re.findall(r'\d+', age_info)
    
    # Convert the numerical values to integers
    ages = [int(age) for age in ages if age]  # Filter out empty strings
    
    # Check if there are any age values
    if ages:
        # If there are age values, join them with "years |" separator
        formatted_ages = " | ".join(f"{age} months" for age in ages)
    else:
        # If no age values are found, return the original age_info
        formatted_ages = age_info
    
    return formatted_ages

In [26]:
df["formatted_age"] = month_df["AGE"].apply(convert_age_format_months)

In [27]:
df

,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT,formatted_age
0,0,NCT05664971,JS004 Combined With Toripalimab and With Stand...,https://www.clinicaltrials.gov/study/NCT05664971,Recruiting,Advanced Lung Cancer,[{'Location Facility': 'Shanghai Chest Hospita...,"February 9, 2023","November 17, 2024",18 Years||Adult|Older Adult,Eligible participants include males or females...,The primary outcomes include the incidence of ...,Secondary outcomes encompass the duration of r...,The study involves multiple cohorts receiving ...,"{'Name': None, 'Organization': None, 'Email': ...",NaN
1,1,NCT05902520,Adoptive Cell Therapy Using Cancer Specific CD...,https://www.clinicaltrials.gov/study/NCT05902520,Recruiting,"HNSCC, Melanoma, Gynecologic Cancer, Colorecta...",[{'Location Facility': 'Providence Portland Me...,"June 19, 2023","May 19, 2026",18 Years||Adult|Older Adult,Eligibility includes adults over 18 with histo...,The primary outcome is to assess the safety of...,Secondary outcomes include evaluating the effi...,The study involves two experimental arms. Both...,"{'Name': None, 'Organization': None, 'Email': ...",NaN
2,2,NCT03917043,APG-2449 in Patients With Advanced Solid Tumors,https://www.clinicaltrials.gov/study/NCT03917043,Recruiting,"Advanced Solid Cancer, Non Small Cell Lung Can...",[{'Location Facility': 'Beijing Cancer Hospita...,"May 27, 2019",February 2025,18 Years||Adult|Older Adult,Eligibility includes patients with advanced tu...,The primary outcomes are to determine the maxi...,Secondary outcomes include measuring the maxim...,The intervention involves administering APG-24...,"{'Name': None, 'Organization': None, 'Email': ...",NaN
3,3,NCT05674422,GEMCAD-REVEAL STUDY - Circulating Tumor DNA as...,https://www.clinicaltrials.gov/study/NCT05674422,Recruiting,Patients With Locally Advanced Rectal Cancer,[{'Location Facility': 'Hospital Germans Trias...,"May 26, 2023",July 2026,18 Years||Adult|Older Adult,Eligible participants are legally capable indi...,The primary outcome measure is the detection o...,Secondary outcomes include the classification ...,The intervention involves total neoadjuvant th...,"{'Name': None, 'Organization': None, 'Email': ...",NaN
4,4,NCT06249854,Efficacy and Safety of Combination Therapy Wit...,https://www.clinicaltrials.gov/study/NCT06249854,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Korea University Ansan...,"February 8, 2024","December 31, 2025",19 Years||Adult|Older Adult,Eligibility includes adults aged 19 or older w...,The primary outcome measure is Progression-Fre...,Secondary outcomes include Disease Control Rat...,The study involves two groups: one receiving P...,"{'Name': None, 'Organization': None, 'Email': ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,3831,NCT05006131,Pancreatic Cancer Screening for At-risk Indivi...,https://www.clinicaltrials.gov/study/NCT05006131,Recruiting,"Pancreatic Cancer, Adult",[{'Location Facility': 'Beth Israel Deaconess ...,"July 10, 2020","July 10, 2026",18 Years|90 Years|Adult|Older Adult,Eligibility includes individuals indicated for...,The primary outcome is the proportion of scree...,Secondary outcomes include the proportion of s...,The intervention involves screening for pancre...,"{'Name': None, 'Organization': None, 'Email': ...",NaN
3832,3832,NCT04679064,Trial on NIraparib-TSR-042 (Dostarlimab) vs Ph...,https://www.clinicaltrials.gov/study/NCT04679064,Recruiting,Ovarian Cancer,[{'Location Facility': 'Azienda Ospedaliera Sp...,"December 1, 2020","January 1, 2025",18 Years||Adult|Older Adult,"Eligibility includes recurrent ovarian, Fallop...",The primary outcome measure is overall surviva...,Secondary outcomes include progression-free su...,Interventions include a choice of standard che...,"{'Name': None, 'Organization': None, 'Email': ...",NaN
3833,3833

In [28]:
df["formatted_age"] = df['AGE'].apply(convert_age_format)

In [29]:
df["formatted_age"]

0                        18 years
1                        18 years
2                        18 years
3                        18 years
4                        19 years
                  ...            
3831          18 years | 90 years
3832                     18 years
3833                     30 years
3834    ||Child|Adult|Older Adult
3835          18 years | 89 years
Name: formatted_age, Length: 3836, dtype: object

In [30]:
df["AGE"]

0               18 Years||Adult|Older Adult
1               18 Years||Adult|Older Adult
2               18 Years||Adult|Older Adult
3               18 Years||Adult|Older Adult
4               19 Years||Adult|Older Adult
                       ...                 
3831    18 Years|90 Years|Adult|Older Adult
3832            18 Years||Adult|Older Adult
3833            30 Years||Adult|Older Adult
3834              ||Child|Adult|Older Adult
3835    18 Years|89 Years|Adult|Older Adult
Name: AGE, Length: 3836, dtype: object

In [31]:
def has_numerical_age(age_info):
    ages = re.findall(r'\d+', age_info)
    return bool(ages) 

In [32]:
df[df['AGE'].apply(has_numerical_age) == False]

,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT,formatted_age
35,35,NCT05364138,Evaluating Scaleup of an Adapted Breast Cancer...,https://www.clinicaltrials.gov/study/NCT05364138,Recruiting,"Breast Cancer, Benign Breast Disease, Cervix C...",[{'Location Facility': 'Partners in Health (In...,"July 31, 2023","March 30, 2024",||Child|Adult|Older Adult,Inclusion criteria include all individuals eva...,The primary outcome focuses on the number of p...,Secondary outcomes include analyzing the media...,The document does not specify interventions di...,"{'Name': None, 'Organization': None, 'Email': ...",||Child|Adult|Older Adult
115,115,NCT05834686,HER2 Expression of CTC to Predict Response in ...,https://www.clinicaltrials.gov/study/NCT05834686,Recruiting,Breast Cancer,[{'Location Facility': 'The Fifth Medical Cent...,"January 1, 2023",April 2025,||Child|Adult|Older Adult,Eligibility includes individuals with patholog...,The primary outcome measure is Progression Fre...,Secondary outcomes include the Objective Respo...,The intervention involves detecting HER2 expre...,"{'Name': None, 'Organization': None, 'Email': ...",||Child|Adult|Older Adult
132,132,NCT05770544,DETERMINE Trial Treatment Arm 03: Entrectinib ...,https://www.clinicaltrials.gov/study/NCT05770544,Recruiting,"Solid Tumor, Haematological Malignancy, Malign...",[{'Location Facility': 'Belfast City Hospital'...,June 2024,October 2029,||Child|Adult|Older Adult,"Eligibility includes adults, teenagers, and ch...",The primary outcomes focus on Objective Respon...,Secondary outcomes include Duration of Respons...,The intervention involves administering entrec...,"{'Name': None, 'Organization': None, 'Email': ...",||Child|Adult|Older Adult
227,227,NCT05770102,DETERMINE Trial Treatment Arm 02: Atezolizumab...,https://www.clinicaltrials.gov/study/NCT05770102,Recruiting,"Malignancy, Malignant Neoplasm, Lymphoprolifer...",[{'Location Facility': 'Belfast City Hospital'...,"October 25, 2023",October 2029,||Child|Adult|Older Adult,Participants must fulfill eligibility criteria...,The primary outcomes include Objective Respons...,Secondary outcomes encompass Duration of Respo...,The intervention involves administering atezol...,"{'Name': None, 'Organization': None, 'Email': ...",||Child|Adult|Older Adult
242,242,NCT05949021,OCTANE: Adjuvant Liposomal Doxorubicin and Car...,https://www.clinicaltrials.gov/study/NCT05949021,Recruiting,Triple Negative Breast Cancer (TNBC),[{'Location Facility': 'RWJBarnabas Health - T...,"October 7, 2023","September 30, 2027",||Child|Adult|Older Adult,Eligibility includes newly diagnosed early-sta...,The primary outcome is the Disease-Free Surviv...,Secondary outcomes focus on the safety and tox...,The intervention involves a combination of lip...,"{'Name': None, 'Organization': None, 'Email': ...",||Child|Adult|Older Adult
296,296,NCT05851677,RC48-ADC in Breast Cancer,https://www.clinicaltrials.gov/study/NCT05851677,Recruiting,Breast Cancer,"[{'Location Facility': 'Renji Hospital, School...","June 1, 2023",August 2027,||Child|Adult|Older Adult,Participants must have histologically confirme...,The primary outcome measure is Progression-Fre...,The secondary outcome focuses on the assessmen...,The intervention involves administering the dr...,"{'Name': None, 'Organization': None, 'Email': ...",||Child|Adult|Older Adult
426,426,NCT04660435,To Identify Primary Resistance to CDK4/6 Inhib...,https://www.clinicaltrials.gov/study/NCT04660435,Recruiting,"Metastatic Breast Cancer, ER Positive Breast C...",[{'Location Facility': 'Hospital Santo Stefano...,"September 1, 2020","December 1, 2026",||Child|Adult|Older Adult,"Women aged 18 or older with ER-positive, HER2-...",The primary outcome measure is progression-fre...,The secondary outcome measure is the clinical ...,The intervention involves a diagnostic test ca...,"{'

In [33]:
df

,Unnamed: 0,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,LOCATIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,POINT_OF_CONTACT,formatted_age
0,0,NCT05664971,JS004 Combined With Toripalimab and With Stand...,https://www.clinicaltrials.gov/study/NCT05664971,Recruiting,Advanced Lung Cancer,[{'Location Facility': 'Shanghai Chest Hospita...,"February 9, 2023","November 17, 2024",18 Years||Adult|Older Adult,Eligible participants include males or females...,The primary outcomes include the incidence of ...,Secondary outcomes encompass the duration of r...,The study involves multiple cohorts receiving ...,"{'Name': None, 'Organization': None, 'Email': ...",18 years
1,1,NCT05902520,Adoptive Cell Therapy Using Cancer Specific CD...,https://www.clinicaltrials.gov/study/NCT05902520,Recruiting,"HNSCC, Melanoma, Gynecologic Cancer, Colorecta...",[{'Location Facility': 'Providence Portland Me...,"June 19, 2023","May 19, 2026",18 Years||Adult|Older Adult,Eligibility includes adults over 18 with histo...,The primary outcome is to assess the safety of...,Secondary outcomes include evaluating the effi...,The study involves two experimental arms. Both...,"{'Name': None, 'Organization': None, 'Email': ...",18 years
2,2,NCT03917043,APG-2449 in Patients With Advanced Solid Tumors,https://www.clinicaltrials.gov/study/NCT03917043,Recruiting,"Advanced Solid Cancer, Non Small Cell Lung Can...",[{'Location Facility': 'Beijing Cancer Hospita...,"May 27, 2019",February 2025,18 Years||Adult|Older Adult,Eligibility includes patients with advanced tu...,The primary outcomes are to determine the maxi...,Secondary outcomes include measuring the maxim...,The intervention involves administering APG-24...,"{'Name': None, 'Organization': None, 'Email': ...",18 years
3,3,NCT05674422,GEMCAD-REVEAL STUDY - Circulating Tumor DNA as...,https://www.clinicaltrials.gov/study/NCT05674422,Recruiting,Patients With Locally Advanced Rectal Cancer,[{'Location Facility': 'Hospital Germans Trias...,"May 26, 2023",July 2026,18 Years||Adult|Older Adult,Eligible participants are legally capable indi...,The primary outcome measure is the detection o...,Secondary outcomes include the classification ...,The intervention involves total neoadjuvant th...,"{'Name': None, 'Organization': None, 'Email': ...",18 years
4,4,NCT06249854,Efficacy and Safety of Combination Therapy Wit...,https://www.clinicaltrials.gov/study/NCT06249854,Recruiting,Non-small Cell Lung Cancer,[{'Location Facility': 'Korea University Ansan...,"February 8, 2024","December 31, 2025",19 Years||Adult|Older Adult,Eligibility includes adults aged 19 or older w...,The primary outcome measure is Progression-Fre...,Secondary outcomes include Disease Control Rat...,The study involves two groups: one receiving P...,"{'Name': None, 'Organization': None, 'Email': ...",19 years
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831,3831,NCT05006131,Pancreatic Cancer Screening for At-risk Indivi...,https://www.clinicaltrials.gov/study/NCT05006131,Recruiting,"Pancreatic Cancer, Adult",[{'Location Facility': 'Beth Israel Deaconess ...,"July 10, 2020","July 10, 2026",18 Years|90 Years|Adult|Older Adult,Eligibility includes individuals indicated for...,The primary outcome is the proportion of scree...,Secondary outcomes include the proportion of s...,The intervention involves screening for pancre...,"{'Name': None, 'Organization': None, 'Email': ...",18 years | 90 years
3832,3832,NCT04679064,Trial on NIraparib-TSR-042 (Dostarlimab) vs Ph...,https://www.clinicaltrials.gov/study/NCT04679064,Recruiting,Ovarian Cancer,[{'Location Facility': 'Azienda Ospedaliera Sp...,"December 1, 2020","January 1, 2025",18 Years||Adult|Older Adult,"Eligibility includes recurrent ovarian, Fallop...",The primary outcome measure is overall surviva...,Secondary outcomes include progression-free su...,Interventions include a choice of standard che...,"{'Name': None, 'Organiz

In [43]:
def filter_by_age(age_input):
        output = {"age_min" : "age",
          "age_max" : "age" }
        system_prompt = f"""You are helpful assistant to max age and min age from given senetence and give output in json format as {output} """ 

        response = openai.chat.completions.create(
            model='gpt-4-0125-preview', 
            temperature=0,
            messages=[
                {"role":"system", "content":system_prompt},
                {"role":"user", "content":f""" Extract the max age and min age from given input 18 Years|75 Years|Adult|Older Adult and give output in {output} format. """},
                {"role":"assistant", "content":"""{"age_min" : "18",
          "age_max" : "75" }""" },
          {"role":"user", "content":f""" Extract the max age and min age from given input {age_input} and give output in {output} format. """},
            ],
            max_tokens = 1024,
            response_format={ "type": "json_object" }
            
        )  
        res = response.choices[0].message.content
        return res

In [44]:
df["formatted_age"] = df["AGE"].apply(filter_by_age)

In [34]:
output = {"age_min" : "age",
          "age_max" : "age" }
system_prompt = f"""You are helpful assistant to max age and min age from given senetence and give output in json format as {output} """        

In [35]:
age_input = df["AGE"][6]


In [36]:
response = openai.chat.completions.create(
            model='gpt-4-0125-preview', 
            temperature=0,
            messages=[
                {"role":"system", "content":system_prompt},
                {"role":"user", "content":f""" Extract the max age and min age from given input 18 Years|75 Years|Adult|Older Adult and give output in {output} format. """},
                {"role":"assistant", "content":"""{"age_min" : "18",
          "age_max" : "75" }""" },
          {"role":"user", "content":f""" Extract the max age and min age from given input {age_input} and give output in {output} format. """},
            ],
            max_tokens = 1024,
            response_format={ "type": "json_object" }
            
        )
      

In [37]:
response.choices[0].message.content

'{"age_min": "18", "age_max": "75"}'